In [1]:
import os
import string
import tempfile
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.python.keras.preprocessing import sequence
from tensorboard import summary as summary_lib
from sklearn.model_selection import train_test_split

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

1.13.1


In [25]:
from pymystem3 import Mystem
from tqdm import tqdm_notebook,tqdm_pandas,tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import KeyedVectors

import nltk

import tf_metrics
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [4]:
tqdm_pandas(tqdm)

## Загрузка данных


In [5]:
df_train = pd.read_csv('60k-classes-text-classification/train.csv',sep=',')
df_test = pd.read_csv('60k-classes-text-classification/test.csv',sep=',')

In [34]:
df_train

,index,text,labels
0,0,Astrology: I am a Capricorn Sun Cap moon and c...,0
1,1,"I'm a triple Capricorn (Sun, Moon and ascendan...",0
2,2,How can I be a good geologist?,1
3,3,What should I do to be a great geologist?,1
4,4,How do I read and find my YouTube comments?,2
5,5,How can I see all my Youtube comments?,2
6,6,What can make Physics easy to learn?,3
7,7,How can you make physics easy to learn?,3
8,8,What was your first sexual experience like?,4
9,9,What was your first sexual experience?,4


In [12]:
df_train[df_train['labels']!=-1]['labels'].value_counts().count()

60416

In [15]:
df_train['text'].apply(len).median()

48.0

In [23]:
def text_prep_tags(text):
    
    text = text.lower()
    result = []
    # Убираем лишние символы
    #text = re.sub(r'[;,]',r' ',text).strip()
    text = re.sub(r'[^\w\s\.]',r'',text).strip()   
    #text = [token.text for token in razdel.tokenize(text)]
    # Делаем лемматизацию       
#     text = [lemma for lemma in mystem.lemmatize(text) if not lemma.isspace() and lemma not in sw_ru
#             and lemma.strip() not in ['.','..','...']]
    
    
    for item in mystem.analyze(text):
      #  print(item)
        token = None
        if item.get('analysis'):
            lemma = item['analysis'][0]['lex']
            pos = re.split('[=,]', item['analysis'][0]['gr'])[0]
            #and lemma not in sw_ru
            if not lemma.isspace()  and lemma.strip() not in ['.','..','...'] and lemma not in sw_ru:
     
                token = f'{lemma}_{upos_map[pos]}'
        else:
            lem_text = item["text"]
            if not lem_text.isspace() and lem_text.strip() not in ['.','..','...'] and lem_text not in sw_ru:
            
                token = f'{lem_text}_UNKN'
            
        if token:    
            result.append(token)

    
    
    
    # Лемматизатор неправильно разбивает слова с дефисом, поэтому исправляем это
#     if '-' in text:
#         for l in range(len(text)):
#             if text[l] == '-':
#                 text[l] = f'{text[l-1]}-{text[l+1]}'
#                 text[l-1] = text[l+1] = text[l]
    
    return result

In [18]:
mystem = Mystem()

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dmitriy/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
from nltk.corpus import stopwords
sw_ru = set(stopwords.words('russian'))

In [20]:
upos_map = {'A':'ADJ','ADV':'ADV','ADVPRO':'ADV','ANUM':'ADJ','APRO':'DET','COM':'ADJ','CONJ':'SCON','INTJ':'INTJ','NONLEX':'X','NUM':'NUM','PART':'PART','PR':'ADP','S':'NOUN','SPRO':'PRON','UNKN':'X' ,'V':'VERB'}

In [21]:
# http://vectors.nlpl.eu/repository/11/185.zip
word2vec = KeyedVectors.load_word2vec_format('../model.bin', binary=True)

In [30]:
count_v = CountVectorizer(tokenizer=text_prep_tags)
count_v.fit(tqdm_notebook(pd.concat([df_train['text'],df_test['text']])))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function text_prep_tags at 0x1a5161f730>,
        vocabulary=None)

In [31]:
train_token_text = df_train['text'].progress_apply(text_prep_tags)
test_token_text = df_test['text'].progress_apply(text_prep_tags)

100%|██████████| 58788/58788 [00:09<00:00, 6376.31it/s]


In [44]:
train_ids = train_token_text.progress_apply(lambda x: list(map(lambda y: count_v.vocabulary_[y] +1 , x)))
test_ids = test_token_text.progress_apply(lambda x: list(map(lambda y: count_v.vocabulary_[y] +1 , x)))

100%|██████████| 58788/58788 [00:00<00:00, 135231.56it/s]


In [47]:
vocab_size = len(count_v.vocabulary_)
sentence_size = 60
embedding_size = 300


# первые индексы в словаре зарезервированы за специальными токенами 
# для паддинга, начала предложения, и для слов которые не вошли в словарь
pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2



print("Pad sequences (samples x time)")
train_ids = sequence.pad_sequences(train_ids, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

print("x_train shape:", train_ids.shape)
test_ids = sequence.pad_sequences(test_ids, 
                                 maxlen=sentence_size,
                                 truncating='post',
                                 padding='post',
                                 value=pad_id)

print("x_train shape:", test_ids.shape)



Pad sequences (samples x time)
x_train shape: (150832, 60)
x_train shape: (58788, 60)


In [48]:
train_ids

array([[ 5241, 22539,  3913, ...,     0,     0,     0],
       [22876,  2472, 46991, ...,     0,     0,     0],
       [22149,  8670, 22539, ...,     0,     0,     0],
       ...,
       [22149, 14736,  3987, ...,     0,     0,     0],
       [22149, 14685, 49656, ...,     0,     0,     0],
       [50055, 41721, 22539, ...,     0,     0,     0]], dtype=int32)

In [ ]:
X_train_ids,X_val_ids,y_train,y_val = train_test_split(train_ids,X_train['label'],test_size=0.3)

In [59]:
df_train_triplet = df_train.copy()
df_train_triplet['anchor'] = pd.Series(train_ids.tolist())


In [60]:
df_train_triplet.drop(['index','text'],inplace=True,axis=1)

In [67]:
df_train_triplet_classes = df_train_triplet[df_train_triplet['labels']!=-1]
df_train_triplet_1 = df_train_triplet[df_train_triplet['labels']==-1]

In [69]:
df_train_triplet_classes.shape

(120832, 2)

In [72]:
df_train_triplet_classes = df_train_triplet_classes.reset_index().sort_values(['labels','index'])

In [73]:
df_train_triplet_classes['positive'] = df_train_triplet_classes.reset_index().sort_values(['labels','index'],ascending=[True,False])['tokens'].values

In [75]:
df_train_triplet_classes[df_train_triplet_classes['index']%2==0]

,index,labels,tokens,positive
0,0,0,"[5241, 22539, 3913, 2472, 8799, 44429, 8760, 3...","[22876, 2472, 46991, 8799, 44429, 30178, 4171,..."
2,2,1,"[22149, 8670, 22539, 6401, 2472, 20020, 19578,...","[49919, 41721, 22539, 14685, 46305, 6401, 2472..."
4,4,2,"[22149, 14685, 22539, 37872, 4171, 18027, 3075...","[22149, 8670, 22539, 40860, 3735, 30751, 51013..."
6,6,3,"[49919, 8670, 27974, 34867, 15521, 46305, 2654...","[22149, 8670, 50978, 27974, 34867, 15521, 4630..."
8,8,4,"[49919, 49534, 50997, 18105, 41301, 17154, 269...","[49919, 49534, 50997, 18105, 41301, 17154, 0, ..."
10,10,5,"[49919, 50480, 2472, 47080, 36146, 28874, 1850...","[22149, 50138, 2472, 47080, 36146, 3266, 45781..."
12,12,6,"[49919, 14736, 28162, 28874, 0, 0, 0, 0, 0, 0,...","[49919, 14736, 28162, 28880, 0, 0, 0, 0, 0, 0,..."
14,14,7,"[50055, 4814, 42605, 28230, 37363, 48304, 3573...","[50055, 14685, 34406, 5087, 37363, 37305, 4998..."
16,16,8,"[50055, 14685, 39437, 27369, 50008, 0, 0, 0, 0...","[50055, 4814, 39437, 4171, 7577, 33634, 50008,..."
18,18,9,"[22149, 41721, 22539, 36088, 18507, 8470, 1800...","[22149, 32789, 41721, 25863, 45768, 21520, 109..."
